In [1]:
import pandas as pd
import math
import sys
import ndjson
from text_preparation import tokenize, lemmatize
from collections import defaultdict
from collections import Counter

df = pd.read_csv("./data/prepared_dataset.csv", index_col = 0)
df = df.iloc[:3]
df = tokenize(df)
df = lemmatize(df)


In [3]:
const_size_in_bytes = 1024*1024*100 # type:int

In [43]:
def write_dict_to_file(file_number:int, dict_for_index:defaultdict(list)):    
    for key in dict_for_index.keys():
        term_freqs = (Counter(dict_for_index[key]))
        dict_for_index[key] = term_freqs
    dict_for_index = sorted(dict_for_index.items())
    with open('data/index_blocks/index_file{}.txt'.format(str(file_number)), 'w') as f:
        ndjson.dump(dict_for_index,f)
        file_number+=1
    return file_number  

In [5]:
def SPIMI_invert(block:pd.DataFrame(columns = ['Text']), file_number:int):
    dict_for_index = defaultdict(list)
    for docID, terms_list in block.iterrows():
        for term in terms_list.values[0]:
            if (sys.getsizeof(dict_for_index) <  const_size_in_bytes): #1024*1024*5): #while size of dict is less than 5Mb
                dict_for_index[term].append(docID)
            else:
                file_number = write_dict_to_file(file_number, dict_for_index)
                dict_for_index = defaultdict(list)
    file_number = write_dict_to_file(file_number, dict_for_index)
    return file_number

In [44]:
block_size = 2 # type:int
file_number = 1 # type:int

for block_num in range(math.ceil(df.shape[0]/block_size)):
    block = df.iloc[block_num*block_size:(block_num+1)*block_size]
    file_number = SPIMI_invert(block, file_number)

In [ ]:
#сделать очередь с приоритетом
#берем min term
#для этого терма делаем слияние (не забыть про частоты термов)
#записываем его на диск и удаляем из памяти
#читаем новые строки (по числу свободной памяти? или из этих файлов из которых слияние?
#может быть сделать генератор чтения из файлов и не забыть про EOF (генератор работает пока не дойдет до этого моменты)

In [61]:
files = [open('data/index_blocks/index_file{}.txt'.format(str(i)), 'r') for i in range(1,file_number)]
readed_dict = defaultdict(list)
updating = True
try:
    while updating:
        for file in files:  
            k,v = ndjson.loads(file.readline())[0]
            print(type(v))
            readed_dict[k].append(v)
            print(readed_dict)
            print("------------------")
finally:
    for fh in files:
        fh.close()

<class 'dict'>
defaultdict(<class 'list'>, {'a': [{'0': 6, '1': 5}]})
------------------
<class 'dict'>
defaultdict(<class 'list'>, {'a': [{'0': 6, '1': 5}, {'2': 10}]})
------------------
